In [1]:
#from cassandra.cluster import Cluster
import pandas as pd 
import collections
import numpy as np

In [2]:
import csv

col_list = ["movieId", "title","genres"]

df = pd.read_csv("/home/spetz/Downloads/archive/movie.csv", usecols=col_list)
#file.readlines()
for_query2=df
df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [3]:
df=df.drop(columns=["genres"])


In [4]:
columns_list = ["userId", "movieId","rating","timestamp"]
dx=pd.read_csv("/home/spetz/Downloads/archive/rating.csv", usecols=columns_list)
dx=dx.sort_values("movieId")
dx=dx[:10000000]
dx

,userId,movieId,rating,timestamp
14162882,97809,1,3.0,2008-06-11 04:47:11
15347420,106140,1,5.0,2013-01-29 03:33:49
15347355,106138,1,3.0,2002-07-31 15:48:53
10170195,70354,1,4.5,2011-02-13 18:55:40
10170255,70355,1,3.5,2008-01-26 16:56:54
...,...,...,...,...
7816716,53914,2167,4.0,2003-04-09 00:25:00
8114556,55895,2167,4.0,2005-01-30 21:04:59
10750246,74315,2167,2.0,2000-04-05 02:33:11
19621413,135822,2167,1.0,2000-08-02 02:31:03


In [5]:
dx=dx.drop(columns=["userId"])
dx


,movieId,rating,timestamp
14162882,1,3.0,2008-06-11 04:47:11
15347420,1,5.0,2013-01-29 03:33:49
15347355,1,3.0,2002-07-31 15:48:53
10170195,1,4.5,2011-02-13 18:55:40
10170255,1,3.5,2008-01-26 16:56:54
...,...,...,...
7816716,2167,4.0,2003-04-09 00:25:00
8114556,2167,4.0,2005-01-30 21:04:59
10750246,2167,2.0,2000-04-05 02:33:11
19621413,2167,1.0,2000-08-02 02:31:03


In [6]:
new_list = list()
dicts={}
dxx = dx[(dx['timestamp'] > '2015-01-01') & (dx['timestamp'] <= '2015-01-15')]
for i in range(len(dxx)):
    ar = dxx.loc[(dx['movieId'] == i ) & (dx['rating'] <= 10)]
    rating=ar.apply("rating").sum()
    movies_num=ar['movieId'].value_counts()
    x=sum(rating/movies_num)
    new_list.append(x)
    first_value = dxx['movieId'].iat[i]
    dicts[i] = new_list[i]




In [7]:
dicts={x:y for x,y in dicts.items() if y!=0}
#ERWTIMA3
def compare_dict_values(dictionary, given_float):
    return {i: dictionary[i] for i in dictionary if dictionary[i] > given_float}


final_table=compare_dict_values(dicts,4.4)
final_table = pd.DataFrame(list(final_table.items()), columns=['MovieId', 'Rating'])
final_table=final_table.head(30)
final_table

,MovieId,Rating
0,45,4.500000
1,122,5.000000
2,203,5.000000
3,222,5.000000
4,233,4.500000
5,242,4.500000
6,363,5.000000
7,417,4.500000
8,421,4.500000
9,479,5.000000


In [8]:
df

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
27273,131254,Kein Bund für's Leben (2007)
27274,131256,"Feuer, Eis & Dosenbier (2002)"
27275,131258,The Pirates (2014)
27276,131260,Rentun Ruusu (2001)


In [9]:
Query1 = pd.DataFrame(list(dicts.items()), columns=['movieId', 'rating'])

Query1

,movieId,rating
0,1,3.652174
1,2,2.924242
2,3,2.833333
3,4,4.000000
4,5,2.785714
...,...,...
1111,2160,3.638889
1112,2161,3.583333
1113,2163,0.750000
1114,2166,3.500000


In [10]:
Query1.tail()

,movieId,rating
1111,2160,3.638889
1112,2161,3.583333
1113,2163,0.750000
1114,2166,3.500000
1115,2167,2.941176


In [11]:
df
final_query = Query1.merge(df, on="movieId", how = 'inner')
final_query = final_query.dropna()
final_query


,movieId,rating,title
0,1,3.652174,Toy Story (1995)
1,2,2.924242,Jumanji (1995)
2,3,2.833333,Grumpier Old Men (1995)
3,4,4.000000,Waiting to Exhale (1995)
4,5,2.785714,Father of the Bride Part II (1995)
...,...,...,...
1111,2160,3.638889,Rosemary's Baby (1968)
1112,2161,3.583333,"NeverEnding Story, The (1984)"
1113,2163,0.750000,Attack of the Killer Tomatoes! (1978)
1114,2166,3.500000,Return to Paradise (1998)


In [12]:
#query1
final_query.tail()


,movieId,rating,title
1111,2160,3.638889,Rosemary's Baby (1968)
1112,2161,3.583333,"NeverEnding Story, The (1984)"
1113,2163,0.750000,Attack of the Killer Tomatoes! (1978)
1114,2166,3.500000,Return to Paradise (1998)
1115,2167,2.941176,Blade (1998)


In [13]:
#QUERYY222222222222222222222
for_query2


final_query2 = for_query2.merge(Query1, on="movieId", how = 'inner')
final_query2


,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.652174
1,2,Jumanji (1995),Adventure|Children|Fantasy,2.924242
2,3,Grumpier Old Men (1995),Comedy|Romance,2.833333
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4.000000
4,5,Father of the Bride Part II (1995),Comedy,2.785714
...,...,...,...,...
1111,2160,Rosemary's Baby (1968),Drama|Horror|Thriller,3.638889
1112,2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,3.583333
1113,2163,Attack of the Killer Tomatoes! (1978),Comedy|Horror,0.750000
1114,2166,Return to Paradise (1998),Crime|Drama|Romance|Thriller,3.500000


In [14]:
final_query2=final_query2.dropna()
final_query2


,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.652174
1,2,Jumanji (1995),Adventure|Children|Fantasy,2.924242
2,3,Grumpier Old Men (1995),Comedy|Romance,2.833333
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4.000000
4,5,Father of the Bride Part II (1995),Comedy,2.785714
...,...,...,...,...
1111,2160,Rosemary's Baby (1968),Drama|Horror|Thriller,3.638889
1112,2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,3.583333
1113,2163,Attack of the Killer Tomatoes! (1978),Comedy|Horror,0.750000
1114,2166,Return to Paradise (1998),Crime|Drama|Romance|Thriller,3.500000


In [15]:
#Query2
col_list=['userId','movieId','tag','timestamp']
dyy = pd.read_csv("/home/spetz/Downloads/archive/tag.csv", usecols=col_list)


In [16]:
dyy=dyy.drop(columns=['userId','timestamp'])
dyy
final_query22 = final_query2.merge(dyy, on="movieId", how = 'inner')

In [17]:
final_query22
#final_query22.dropna()


,movieId,title,genres,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.652174,Watched
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.652174,computer animation
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.652174,Disney animated feature
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.652174,Pixar animation
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.652174,TÃ©a Leoni does not star in this movie
...,...,...,...,...,...
97822,2167,Blade (1998),Action|Horror|Thriller,2.941176,Netflix Finland
97823,2167,Blade (1998),Action|Horror|Thriller,2.941176,action
97824,2167,Blade (1998),Action|Horror|Thriller,2.941176,Bibliothek
97825,2167,Blade (1998),Action|Horror|Thriller,2.941176,vampire


In [18]:
#QUERY3
final_query3=final_query2.drop(columns=["movieId",'genres','rating'])

In [19]:
final_query3

,title
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
...,...
1111,Rosemary's Baby (1968)
1112,"NeverEnding Story, The (1984)"
1113,Attack of the Killer Tomatoes! (1978)
1114,Return to Paradise (1998)


In [20]:
only_numbers=final_query3['title'].str.extract('(\d+)')
only_numbers.columns=['years']


In [21]:
only_numbers
final_query2['years']= only_numbers['years']
final_query2
query3=final_query2
query3



,movieId,title,genres,rating,years
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.652174,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,2.924242,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,2.833333,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4.000000,1995
4,5,Father of the Bride Part II (1995),Comedy,2.785714,1995
...,...,...,...,...,...
1111,2160,Rosemary's Baby (1968),Drama|Horror|Thriller,3.638889,1968
1112,2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,3.583333,1984
1113,2163,Attack of the Killer Tomatoes! (1978),Comedy|Horror,0.750000,1978
1114,2166,Return to Paradise (1998),Crime|Drama|Romance|Thriller,3.500000,1998


In [22]:
#query4
query4=final_query22
query5=final_query22

In [23]:

final_query.to_csv("/home/spetz/Downloads/archive/query1.csv")
final_query22.to_csv("/home/spetz/Downloads/archive/query2.csv")
query3.to_csv("/home/spetz/Downloads/archive/query3.csv")
query4.to_csv("/home/spetz/Downloads/archive/query4.csv")
query5.to_csv("/home/spetz/Downloads/archive/query5.csv")

In [24]:
#CASSANDRA 

#CREATE KEYSPACE bigdata
#WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3};


#CREATE TABLE query1(
        #Unnamed int,
        #movieId int,
        #rating float,
        #title text,
        #PRIMARY KEY(movieId,title));

#CREATE TABLE query2(
        #Unnamed int,
        #movieId int,
        #title text,
        #genres text,
        #rating float,
        #tag text,
        #PRIMARY KEY(movieId,title));



#COPY query1(Unnamed,movieId,rating, title) FROM '/home/spetz/Downloads/archive/query1.csv' WITH HEADER = true;
#COPY query2(Unnamed,movieId,title, genres,rating,tag) FROM '/home/spetz/Downloads/archive/query2.csv' WITH HEADER = true;






In [27]:
from cassandra.cluster import Cluster
cluster =Cluster(['0.0.0.0'], port=9042)
session = cluster.connect("bigdata")